# 데이터 작업

데이터는 기계 학습 모델 작성 과정의 토대가 되는 요소라 할 수 있습니다. 전문 데이터 과학 솔루션을 빌드하려면 클라우드에서 중앙 집중식으로 데이터를 관리하고, 여러 워크스테이션과 컴퓨팅 대상에서 실험 실행 및 모델 학습을 진행하는 데이터 과학자 팀에게 해당 데이터 액세스 권한을 제공해야 합니다.

이 Notebook에서는 데이터 작업을 위한 두 개의 Azure Machine Learning 개체인 *데이터 저장소* 및 *데이터 자산을 살펴봅니다*.

## 시작하기 전에

이 Notebook에서 코드를 실행하려면 **최신 버전의 azureml-ai-ml** 패키지가 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

> **고**:
> **azure-ai-ml** 패키지가 설치되지 않은 경우 를 실행 `pip install azure-ai-ml` 하여 설치합니다.

In [ ]:
## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

작업 영역에 연결하려면 식별자 매개 변수(구독 ID, 리소스 그룹 이름 및 작업 영역 이름)가 필요합니다. 리소스 그룹 이름 및 작업 영역 이름이 이미 채워져 있습니다. 명령을 완료하려면 구독 ID만 필요합니다.

필요한 매개 변수를 찾으려면 Studio의 오른쪽 위에 있는 구독 및 작업 영역 이름을 클릭합니다. 오른쪽에 창이 열립니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 구독 ID를 복사하고 **YOUR-SUBSCRIPTION-ID** 를 복사한 값으로 바꿉니다. </p>

## 데이터 저장소 나열

Azure Machine Learning 작업 영역을 만들면 Azure Storage 계정도 만들어집니다. 스토리지 계정에는 Blob 및 파일 스토리지가 포함되며 작업 영역과 데이터 저장소로 자동으로 연결 **됩니다**. 작업 영역에 연결된 모든 데이터 저장소를 나열할 수 있습니다.

In [ ]:
`workspaceblobstore` 앞에서 탐색한 **azureml-blobstore-...** 컨테이너에 연결하는 를 확인합니다. 는 `workspacefilestore` **코드-...** 파일 공유에 연결합니다.

In [ ]:
## 데이터 저장소 만들기

다른 Azure Storage 서비스를 Azure Machine Learning 작업 영역에 연결하려는 경우 데이터 저장소를 만들 수 있습니다. 데이터 저장소를 만들면 작업 영역과 스토리지 간에 연결이 만들어지고 스토리지 서비스 자체가 만들어지지 않습니다. 

데이터 저장소를 만들고 (이미 기존) 스토리지에 연결하려면 다음을 지정해야 합니다.

- 연결할 스토리지 서비스의 유형을 나타내는 클래스입니다. 아래 예제에서는 Blob Storage(`AzureBlobDatastore`)에 연결합니다.
- `name`: Azure Machine Learning 작업 영역에 있는 데이터 저장소의 표시 이름입니다.
- `description`: 데이터 저장소에 대한 자세한 정보를 제공하는 선택적 설명입니다.
- `account_name`: Azure Storage 계정의 이름입니다.
- `container_name`: Azure Storage 계정에 Blob을 저장할 컨테이너의 이름입니다.
- `credentials`: 인증 방법 및 인증할 자격 증명을 제공합니다. 아래 예제에서는 계정 키를 사용합니다.

**중요**: 
- **YOUR-STORAGE-ACCOUNT-NAME**을 자동으로 생성된 스토리지 계정의 이름으로 바꿉니다. 
- 에 대한 `account_key` **XXXX-XXXX**를 Azure Storage 계정의 계정 키로 바꿉니다. 

[Azure Portal](https://portal.azure.com) 이동하여 계정 키를 검색하고, 스토리지 계정으로 이동하고, **액세스 키** 탭에서 key1 또는 key2의 **키** 값을 복사할 수 있습니다. 

데이터 저장소를 다시 나열하여 라는 `blob_training_data` 새 데이터 저장소가 만들어졌는지 확인합니다.

In [ ]:
## 데이터 자산 만들기

데이터 저장소의 특정 폴더 또는 파일을 가리키려면 데이터 자산을 만들 수 있습니다. 데이터 자산에는 다음 세 가지 유형이 있습니다.

- `URI_FILE` 는 특정 파일을 가리킵니다.
- `URI_FOLDER` 는 특정 폴더를 가리킵니다.
- `MLTABLE` 는 폴더 내에서 하나 이상의 파일을 읽는 방법을 지정하는 MLTable 파일을 가리킵니다.

세 가지 유형의 데이터 자산을 모두 만들어 두 자산의 차이점을 경험하게 됩니다.

데이터 자산을 만들 `URI_FILE` 려면 특정 파일을 가리키는 경로를 지정해야 합니다. 경로는 로컬 경로 또는 클라우드 경로일 수 있습니다.

아래 예제에서는 *로컬* 경로를 참조하여 데이터 자산을 만듭니다. Azure Machine Learning 작업 영역에서 작업할 때 데이터를 항상 사용할 수 있도록 하려면 로컬 파일이 자동으로 기본 데이터 저장소에 업로드됩니다. 이 경우 파일은 `diabetes.csv` **workspaceblobstore** 데이터 저장소의 **LocalUpload** 폴더에 업로드됩니다. 

로컬 파일에서 데이터 자산을 만들려면 다음 셀을 실행합니다.

데이터 자산을 만들 `URI_FOLDER` 려면 특정 폴더를 가리키는 경로를 지정해야 합니다. 경로는 로컬 경로 또는 클라우드 경로일 수 있습니다.

아래 예제에서는 *클라우드* 경로를 참조하여 데이터 자산을 만듭니다. 경로가 아직 존재할 필요가 없습니다. 데이터가 경로에 업로드될 때 폴더가 만들어집니다.

In [ ]:
데이터 자산을 만들 `MLTable` 려면 MLTable 파일이 포함된 폴더를 가리키는 경로를 지정해야 합니다. 경로는 로컬 경로 또는 클라우드 경로일 수 있습니다. 

아래 예제에서는 MLTable 및 CSV 파일이 포함된 *로컬* 경로를 참조하여 데이터 자산을 만듭니다. 

새 데이터 자산이 만들어졌는지 확인하려면 작업 영역의 모든 데이터 자산을 다시 나열할 수 있습니다.

In [ ]:
## Notebook에서 데이터 읽기

처음에는 Notebook의 데이터 자산으로 작업하여 데이터를 탐색하고 기계 학습 모델을 실험할 수 있습니다. `URI_FOLDER` 또는 `URI_FILE` 형식 데이터 자산은 일반적으로 데이터를 읽는 것처럼 읽습니다. 예를 들어 데이터 자산이 가리키는 CSV 파일을 읽으려면 pandas 함수 `read_csv()`를 사용할 수 있습니다. 

`MLTable` 형식 데이터 자산은 스키마 및 데이터 해석 방법을 지정하는 **MLTable** 파일에서 이미 *읽습니다*. 데이터가 이미 *읽혀* 있으므로 MLTable 데이터 자산을 pandas 데이터 프레임으로 쉽게 변환할 수 있습니다. 

터미널에서 했던 라이브러리를 `mltable` 설치해야 합니다. 그런 다음 데이터 자산을 데이터 프레임으로 변환하고 데이터를 시각화할 수 있습니다.  

## 작업에서 데이터 사용

실험에 Notebook을 사용한 후 스크립트를 사용하여 기계 학습 모델을 학습시킬 수 있습니다. 스크립트는 작업으로 실행할 수 있으며 각 작업에 대해 입력 및 출력을 지정할 수 있습니다. 

**데이터 자산** 또는 **데이터 저장소 경로를** 작업의 입력 또는 출력으로 사용할 수 있습니다. 

아래 셀은 **src** 폴더에 **move-data.py** 스크립트를 만듭니다. 스크립트는 함수를 사용하여 입력 데이터를 읽습니다 `read_csv()` . 그런 다음 스크립트는 데이터를 출력 경로에 CSV 파일로 저장합니다.

**move-data.py** 스크립트를 실행하는 작업을 제출하려면 아래 셀을 실행합니다. 

작업은 로컬 **diabetes.csv** 파일을 입력으로 가리키는 데이터 자산을 `diabetes-local`사용하도록 구성됩니다. 출력은 새 데이터 저장소 `blob_training_data`의 폴더를 가리키는 경로입니다.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = './data/diabetes.csv'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Data asset pointing to a local file, automatically uploaded to the default datastore",
    name="diabetes-local"
)

ml_client.data.create_or_update(my_data)

To create a `URI_FOLDER` data asset, you have to specify a path that points to a specific folder. The path can be a local path or cloud path.

In the example below, you'll create a data asset by referencing a *cloud* path. The path doesn't have to exist yet. The folder will be created when data is uploaded to the path.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

datastore_path = 'azureml://datastores/blob_training_data/paths/data-asset-path/'

my_data = Data(
    path=datastore_path,
    type=AssetTypes.URI_FOLDER,
    description="Data asset pointing to data-asset-path folder in datastore",
    name="diabetes-datastore-path"
)

ml_client.data.create_or_update(my_data)

To create a `MLTable` data asset, you have to specify a path that points to a folder which contains a MLTable file. The path can be a local path or cloud path. 

In the example below, you'll create a data asset by referencing a *local* path which contains an MLTable and CSV file. 

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

local_path = 'data/'

my_data = Data(
    path=local_path,
    type=AssetTypes.MLTABLE,
    description="MLTable pointing to diabetes.csv in data folder",
    name="diabetes-table"
)

ml_client.data.create_or_update(my_data)

To verify that the new data assets have been created, you can list all data assets in the workspace again:

In [ ]:
datasets = ml_client.data.list()
for ds_name in datasets:
    print(ds_name.name)

## Read data in notebook

Initially, you may want to work with data assets in notebooks, to explore the data and experiment with machine learning models. Any `URI_FILE` or `URI_FOLDER` type data assets are read as you would normally read data. For example, to read a CSV file a data asset points to, you can use the pandas function `read_csv()`. 

A `MLTable` type data asset is already *read* by the **MLTable** file, which specifies the schema and how to interpret the data. Since the data is already *read*, you can easily convert a MLTable data asset to a pandas dataframe. 

You'll need to install the `mltable` library (which you did in the terminal). Then, you can convert the data asset to a dataframe and visualize the data.  

In [ ]:
import mltable

registered_data_asset = ml_client.data.get(name='diabetes-table', version=1)
tbl = mltable.load(f"azureml:/{registered_data_asset.id}")
df = tbl.to_pandas_dataframe()
df.head(5)

## Use data in a job

After using a notebook for experimentation. You can use scripts to train machine learning models. A script can be run as a job, and for each job you can specify inputs and outputs. 

You can use either **data assets** or **datastore paths** as inputs or outputs of a job. 

The cells below creates the **move-data.py** script in the **src** folder. The script reads the input data with the `read_csv()` function. The script then stores the data as a CSV file in the output path.

In [ ]:
import os

# create a folder for the script files
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

In [ ]:
%%writefile $script_folder/move-data.py
# import libraries
import argparse
import pandas as pd
import numpy as np
from pathlib import Path

def main(args):
    # read data
    df = get_data(args.input_data)

    output_df = df.to_csv((Path(args.output_datastore) / "diabetes.csv"), index = False)

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)

    # Count the rows and print the result
    row_count = (len(df))
    print('Analyzing {} rows of data'.format(row_count))
    
    return df

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--input_data", dest='input_data',
                        type=str)
    parser.add_argument("--output_datastore", dest='output_datastore',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


To submit a job that runs the **move-data.py** script, run the cell below. 

The job is configured to use the data asset `diabetes-local`, pointing to the local **diabetes.csv** file as input. The output is a path pointing to a folder in the new datastore `blob_training_data`.

In [ ]:
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import command

# configure input and output
my_job_inputs = {
    "local_data": Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-local:1")
}

my_job_outputs = {
    "datastore_data": Output(type=AssetTypes.URI_FOLDER, path="azureml://datastores/blob_training_data/paths/datastore-path")
}

# configure job
job = command(
    code="./src",
    command="python move-data.py --input_data ${{inputs.local_data}} --output_datastore ${{outputs.datastore_data}}",
    inputs=my_job_inputs,
    outputs=my_job_outputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="move-diabetes-data",
    experiment_name="move-diabetes-data"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)